# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [193]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [3]:
df_receipt.loc[0:9, ['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [4]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename({'sales_ymd': 'sales_date'}).head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [7]:
df_receipt.loc[df_receipt['customer_id']=='CS018205000001', ['sales_ymd', 'customer_id', 'product_cd', 'amount']]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


In [8]:
# 別解
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [9]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" and amount >= 1000')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [10]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query(
    'customer_id == "CS018205000001" and (amount >= 1000 or quantity >= 5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [12]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query(
    'customer_id == "CS018205000001" and (1000<= amount <= 2000)')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [13]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query(
    'customer_id == "CS018205000001" and product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [14]:
df_store.query('prefecture_cd != "13" and floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [16]:
df_store.query('store_cd.str.startswith("S14")', engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [17]:
df_customer.query('customer_id.str.endswith("1")', engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [18]:
df_store.loc[df_store['address'].str.startswith('神奈川県横浜市')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [19]:
df_customer.query("status_cd.str.contains('^[A-F]')", engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [20]:
df_customer.query("status_cd.str.contains('[1-9]$')", engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [21]:
df_customer.query("status_cd.str.contains('^[A-F].*[1-9]$')", engine='python').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [22]:
df_store.query("tel_no.str.contains('^[0-9]{3}-[0-9]{3}-[0-9]{4}$')", engine='python').head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [23]:
df_customer.sort_values('birth_day').head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [24]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [40]:
pd.concat([df_receipt[['customer_id', 'amount']],
           df_receipt['amount'].rank(method='min', ascending=False).astype('int').rename('ranking')],
          axis=1) \
  .sort_values('ranking') \
  .head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
4632,CS015515000034,5480,4
72747,ZZ000000000000,5480,4
10320,ZZ000000000000,5480,4
97294,CS021515000089,5440,7
28304,ZZ000000000000,5440,7
92246,CS009415000038,5280,9
68553,CS040415000200,5280,9


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [42]:
pd.concat([df_receipt[['customer_id', 'amount']],
           df_receipt['amount'].rank(method='first', ascending=False).astype('int').rename('ranking')],
          axis=1) \
  .sort_values('ranking') \
  .head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
4632,CS015515000034,5480,4
10320,ZZ000000000000,5480,5
72747,ZZ000000000000,5480,6
28304,ZZ000000000000,5440,7
97294,CS021515000089,5440,8
596,CS015515000083,5280,9
11275,CS017414000114,5280,10


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [43]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [44]:
len(df_receipt['customer_id'].unique())

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [45]:
df_receipt[['store_cd', 'amount', 'quantity']].groupby('store_cd').sum()

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [54]:
df_receipt.groupby('customer_id').sales_ymd.max().head(10)

customer_id
CS001113000004    20190308
CS001114000005    20190731
CS001115000010    20190405
CS001205000004    20190625
CS001205000006    20190224
CS001211000025    20190322
CS001212000027    20170127
CS001212000031    20180906
CS001212000046    20170811
CS001212000070    20191018
Name: sales_ymd, dtype: int64

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [55]:
df_receipt.groupby('customer_id').sales_ymd.min().head(10)

customer_id
CS001113000004    20190308
CS001114000005    20180503
CS001115000010    20171228
CS001205000004    20170914
CS001205000006    20180207
CS001211000025    20190322
CS001212000027    20170127
CS001212000031    20180906
CS001212000046    20170811
CS001212000070    20191018
Name: sales_ymd, dtype: int64

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [75]:
df = df_receipt.groupby('customer_id').agg({'sales_ymd': [max, min]}).reset_index()
df.columns = ['customer_id', 'newest', 'oldest']
df[df.newest != df.oldest].head(10)

,customer_id,newest,oldest
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [81]:
df_receipt.groupby('store_cd').amount.mean().sort_values(ascending=False).head(10)

store_cd
S13052    402.867470
S13015    351.111960
S13003    350.915519
S14010    348.791262
S13001    348.470386
S13020    337.879932
S14011    335.718333
S14026    332.340588
S13004    330.943949
S13019    330.208616
Name: amount, dtype: float64

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [82]:
df_receipt.groupby('store_cd').amount.median().sort_values(ascending=False).head(10)

store_cd
S13052    190
S14010    188
S14050    185
S14040    180
S13003    180
S13018    180
S14033    179
S13008    179
S13035    178
S13016    178
Name: amount, dtype: int64

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [87]:
df_receipt.groupby('store_cd')['product_cd'].apply(lambda x: x.mode()).head()

store_cd   
S12007    0    P060303001
S12013    0    P060303001
S12014    0    P060303001
S12029    0    P060303001
S12030    0    P060303001
S13001    0    P060303001
S13002    0    P060303001
S13003    0    P071401001
S13004    0    P060303001
S13005    0    P040503001
S13008    0    P060303001
S13009    0    P060303001
S13015    0    P071401001
S13016    0    P071102001
S13017    0    P060101002
S13018    0    P071401001
S13019    0    P071401001
S13020    0    P071401001
S13031    0    P060303001
S13032    0    P060303001
S13035    0    P040503001
S13037    0    P060303001
S13038    0    P060303001
S13039    0    P071401001
S13041    0    P071401001
S13043    0    P060303001
S13044    0    P060303001
S13051    0    P050102001
          1    P071003001
          2    P080804001
S13052    0    P050101001
S14006    0    P060303001
S14010    0    P060303001
S14011    0    P060101001
S14012    0    P060303001
S14021    0    P060101001
S14022    0    P060303001
S14023    0    P071401001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [103]:
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [105]:
df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index().sort_values('amount', ascending=False).head()

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [110]:
df_receipt['amount'].quantile([0, 0.25, 0.5, 0.75, 1])

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

In [111]:
df_receipt['amount'].quantile(np.arange(0, 1.1, 0.25))

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [115]:
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [121]:
df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [3]:
pd.merge(df_receipt, df_store[['store_cd','store_name']], how='inner', on='store_cd').head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1258588800,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1228953600,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1287619200,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1215648000,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1280966400,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [5]:
pd.merge(df_product, df_category[['category_small_cd','category_small_name']], how='inner', on='category_small_cd').head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [52]:
pd.merge(df_customer, df_receipt, how='left', on='customer_id') \
.query('(gender_cd == "1") and (not customer_id.str.startswith("Z"))', engine='python') \
.fillna(0) \
.groupby('customer_id').sum().reset_index()[['customer_id', 'amount']] \
.head(10)

,customer_id,amount
0,CS001112000009,0.0
1,CS001112000019,0.0
2,CS001112000021,0.0
3,CS001112000023,0.0
4,CS001112000024,0.0
5,CS001112000029,0.0
6,CS001112000030,0.0
7,CS001113000004,1298.0
8,CS001113000010,0.0
9,CS001114000005,626.0


In [54]:
# 別解（解答のやり方）

df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
# print(len(df_amount_sum['customer_id']))
df_tmp = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")', engine='python')
# print(len(df_tmp.groupby('customer_id').sum()))
tmp = pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0)
# print(len(tmp))
tmp.head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


In [53]:
# pandasでは、groupbyの際にデフォルトでインデックスがソートされるので、上と下で結果が違うように見えるのはそのため。
# 上の場合は、df_customerのidがgroupbyの際にソートされるが、下の解法では最終処理のpd.mergeで
# left joinされるDataFrameがdf_customerの元々のidの順番になっている。
# 下の解法と見た目が同じ結果を出すには、上の解法のgroupbyのオプションにおいて、sort=Falseとする。

pd.merge(df_customer, df_receipt, how='left', on='customer_id') \
.query('(gender_cd == "1") and (not customer_id.str.startswith("Z"))', engine='python').fillna(0) \
.groupby('customer_id', sort=False).sum().reset_index()[['customer_id', 'amount']] \
.head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [66]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine='python')

df_amount = df.groupby('customer_id').sum().reset_index()[['customer_id', 'amount']] \
.sort_values('amount', ascending=False) \
.iloc[:20]

df_sales = df[['customer_id', 'sales_ymd']].drop_duplicates().groupby('customer_id').count().reset_index() \
.sort_values('sales_ymd', ascending=False) \
.iloc[:20]

pd.merge(df_sales, df_amount, how='outer', on='customer_id')

,customer_id,sales_ymd,amount
0,CS040214000008,23.0,NaN
1,CS015415000185,22.0,20153.0
2,CS010214000010,22.0,18585.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0
5,CS017415000097,20.0,23086.0
6,CS016415000141,20.0,18372.0
7,CS031414000051,19.0,19202.0
8,CS014214000023,19.0,NaN
9,CS022515000226,19.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [68]:
len(df_store['store_cd'].unique()) * len(df_product['product_cd'].unique())

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [71]:
df = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
df['diff_amount'] = df['amount'].diff()
df.head(10)

,sales_ymd,amount,diff_amount
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
5,20170106,32387,-5443.0
6,20170107,23415,-8972.0
7,20170108,24737,1322.0
8,20170109,26718,1981.0
9,20170110,20143,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [73]:
df = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1, 3+1):
    df['amount_{}'.format(i)] = df['amount'].shift(i)

df.head(10)

,sales_ymd,amount,amount_1,amount_2,amount_3
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,33723.0,NaN,NaN
2,20170103,27503,24165.0,33723.0,NaN
3,20170104,36165,27503.0,24165.0,33723.0
4,20170105,37830,36165.0,27503.0,24165.0
5,20170106,32387,37830.0,36165.0,27503.0
6,20170107,23415,32387.0,37830.0,36165.0
7,20170108,24737,23415.0,32387.0,37830.0
8,20170109,26718,24737.0,23415.0,32387.0
9,20170110,20143,26718.0,24737.0,23415.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [75]:
df = pd.merge(df_receipt, df_customer, how='inner', on='customer_id')
df['age_level'] = df['age'].apply(lambda x: int(x/10)*10)
df_sales_summary = pd.pivot_table(df, index='age_level', columns='gender', values='amount', aggfunc=np.sum)
df_sales_summary

gender,不明,女性,男性
age_level,,,
10,4317.0,149836.0,1591.0
20,44328.0,1363724.0,72940.0
30,50441.0,693047.0,177322.0
40,483512.0,9320791.0,19355.0
50,342923.0,6685192.0,54320.0
60,71418.0,987741.0,272469.0
70,2427.0,29764.0,13435.0
80,5111.0,262923.0,46360.0
90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [80]:
df_sales_summary.stack().reset_index() \
.replace({'不明': '99', '男性': '00', '女性': '01'}) \
.rename(columns={0: 'amount'})

,age_level,gender,amount
0,10,99,4317.0
1,10,01,149836.0
2,10,00,1591.0
3,20,99,44328.0
4,20,01,1363724.0
5,20,00,72940.0
6,30,99,50441.0
7,30,01,693047.0
8,30,00,177322.0
9,40,99,483512.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [92]:
pd.concat([df['customer_id'], df_customer['birth_day'].apply(lambda x: x.strftime('%Y%m%d'))], axis=1).head(10)

,customer_id,birth_day
0,CS006214000001,19810429
1,CS006214000001,19520401
2,CS006214000001,19761004
3,CS006214000001,19330327
4,CS006214000001,19950329
5,CS006214000001,19740915
6,CS006214000001,19770809
7,CS006214000001,19730817
8,CS006214000001,19310502
9,CS006214000001,19620711


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [93]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['application_date'])], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [111]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_ymd'])], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,1970-01-01 00:00:00.020181103
1,1132,2,1970-01-01 00:00:00.020181118
2,1102,1,1970-01-01 00:00:00.020170712
3,1132,1,1970-01-01 00:00:00.020190205
4,1102,2,1970-01-01 00:00:00.020180821
5,1112,1,1970-01-01 00:00:00.020190605
6,1102,2,1970-01-01 00:00:00.020181205
7,1102,1,1970-01-01 00:00:00.020190922
8,1112,2,1970-01-01 00:00:00.020170504
9,1102,1,1970-01-01 00:00:00.020191010


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [107]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009-11-03
1,1132,2,2009-11-18
2,1102,1,2008-07-12
3,1132,1,2010-02-05
4,1102,2,2009-08-21
5,1112,1,2010-06-05
6,1102,2,2009-12-05
7,1102,1,2010-09-22
8,1112,2,2008-05-04
9,1102,1,2010-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [112]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009
1,1132,2,2009
2,1102,1,2008
3,1132,1,2010
4,1102,2,2009
5,1112,1,2010
6,1102,2,2009
7,1102,1,2010
8,1112,2,2008
9,1102,1,2010


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [113]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [114]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [115]:
df = df_receipt[['customer_id', 'amount']].query("not customer_id.str.startswith('Z')", engine='python') \
     .groupby('customer_id').sum().reset_index()
df['flag_amount'] = df['amount'].apply(lambda x: 0 if x <= 2000 else 1)
df.head(10)

,customer_id,amount,flag_amount
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1
5,CS001211000025,456,0
6,CS001212000027,448,0
7,CS001212000031,296,0
8,CS001212000046,228,0
9,CS001212000070,456,0


---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [126]:
df = df_customer[['customer_id', 'postal_cd']].copy()
df['flag_postal'] = df['postal_cd'].apply(lambda x: 1 if 100 <= int(x[:3]) <= 209 else 0)
pd.merge(df, df_receipt[['customer_id']], how='right', on='customer_id') \
.groupby('flag_postal') \
.agg({'customer_id': 'nunique'})

,customer_id
flag_postal,
0.0,3906
1.0,4400


---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [130]:
pd.concat([df_customer[['customer_id', 'address']],
           df_customer['address'].str[:3].replace({'埼玉県': 11, '千葉県':12, '東京都':13, '神奈川':14})],
          axis=1).head(10)

,customer_id,address,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [139]:
df = df_receipt[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
q = np.array(df['amount'].quantile([0.25, 0.5, 0.75, 1.0]))
def q_func(x):
    if x < q[0]:
        return 1
    elif x < q[1]:
        return 2
    elif x < q[2]:
        return 3
    else:
        return 4

df['quantile'] = df['amount'].apply(q_func)
df.head(10)

,customer_id,amount,quantile
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [149]:
df = df_customer.copy()
df['bin_age'] = pd.cut(df['age'], bins=[0,10,20,30,40,50,60,np.inf], labels=['0','10','20','30','40','50','60'], right=False)
df[['customer_id', 'birth_day', 'age', 'bin_age']].head(10)

,customer_id,birth_day,age,bin_age
0,CS021313000114,1981-04-29,37,30
1,CS037613000071,1952-04-01,66,60
2,CS031415000172,1976-10-04,42,40
3,CS028811000001,1933-03-27,86,60
4,CS001215000145,1995-03-29,24,20
5,CS020401000016,1974-09-15,44,40
6,CS015414000103,1977-08-09,41,40
7,CS029403000008,1973-08-17,45,40
8,CS015804000004,1931-05-02,87,60
9,CS033513000180,1962-07-11,56,50


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [159]:
df['gender_age'] = df['gender_cd'] + '-' + df['bin_age'].astype(str)
df[['customer_id', 'gender_cd', 'bin_age', 'gender_age']].head(10)

,customer_id,gender_cd,bin_age,gender_age
0,CS021313000114,1,30,1-30
1,CS037613000071,9,60,9-60
2,CS031415000172,1,40,1-40
3,CS028811000001,1,60,1-60
4,CS001215000145,1,20,1-20
5,CS020401000016,0,40,0-40
6,CS015414000103,1,40,1-40
7,CS029403000008,0,40,0-40
8,CS015804000004,0,60,0-60
9,CS033513000180,1,50,1-50


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [161]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

,customer_id,gender_cd_0,gender_cd_1,gender_cd_9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [163]:
df = df_receipt[['customer_id', 'amount']].query("not customer_id.str.startswith('Z')", engine='python') \
     .groupby('customer_id').sum().reset_index()
df['std_amount'] = (df['amount'] - df['amount'].mean()) / df['amount'].std()
df.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,-0.459350
1,CS001114000005,626,-0.706348
2,CS001115000010,3044,0.182403
3,CS001205000004,1988,-0.205737
4,CS001205000006,3337,0.290096
5,CS001211000025,456,-0.768832
6,CS001212000027,448,-0.771773
7,CS001212000031,296,-0.827641
8,CS001212000046,228,-0.852635
9,CS001212000070,456,-0.768832


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [164]:
df = df_receipt[['customer_id', 'amount']].query("not customer_id.str.startswith('Z')", engine='python') \
     .groupby('customer_id').sum().reset_index()
df['std_amount'] = (df['amount'] - df['amount'].min()) / (df['amount'].max() - df['amount'].min())
df.head(10)

,customer_id,amount,std_amount
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [165]:
df = df_receipt[['customer_id', 'amount']].query("not customer_id.str.startswith('Z')", engine='python') \
     .groupby('customer_id').sum().reset_index()
df['log_amount'] = df['amount'].apply(lambda x: np.log10(x))
df.head(10)

,customer_id,amount,log_amount
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [166]:
df = df_receipt[['customer_id', 'amount']].query("not customer_id.str.startswith('Z')", engine='python') \
     .groupby('customer_id').sum().reset_index()
df['log_amount'] = np.log(df['amount'])
df.head(10)

,customer_id,amount,log_amount
0,CS001113000004,1298,7.168580
1,CS001114000005,626,6.439350
2,CS001115000010,3044,8.020928
3,CS001205000004,1988,7.594884
4,CS001205000006,3337,8.112827
5,CS001211000025,456,6.122493
6,CS001212000027,448,6.104793
7,CS001212000031,296,5.690359
8,CS001212000046,228,5.429346
9,CS001212000070,456,6.122493


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [167]:
df = df_product['unit_price'] - df_product['unit_cost']
df.head()

0    49.0
1    54.0
2    57.0
3    62.0
4    67.0
dtype: float64

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [168]:
df = df_product[['unit_price', 'unit_cost']].dropna()
df['profit_rate'] = (df['unit_price'] - df['unit_cost']) / df['unit_price']
df['profit_rate'].mean()

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [171]:
df = df_product[['unit_price', 'unit_cost']].dropna()
df['new_price'] = (df['unit_cost']/0.7).astype(int)
# df['new_price'] = np.int32(df['unit_cost']/0.7)  # これでもOK
df['profit_rate'] = (df['new_price'] - df['unit_cost']) / df['new_price']
df.head(10)

,unit_price,unit_cost,new_price,profit_rate
0,198.0,149.0,212,0.297170
1,218.0,164.0,234,0.299145
2,230.0,173.0,247,0.299595
3,248.0,186.0,265,0.298113
4,268.0,201.0,287,0.299652
5,298.0,224.0,320,0.300000
6,338.0,254.0,362,0.298343
7,420.0,315.0,450,0.300000
8,498.0,374.0,534,0.299625
9,580.0,435.0,621,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [172]:
df = df_product[['unit_price', 'unit_cost']].dropna()
df['new_price'] = np.round(df['unit_cost']/0.7)
df['profit_rate'] = (df['new_price'] - df['unit_cost']) / df['new_price']
df.head(10)

,unit_price,unit_cost,new_price,profit_rate
0,198.0,149.0,213.0,0.300469
1,218.0,164.0,234.0,0.299145
2,230.0,173.0,247.0,0.299595
3,248.0,186.0,266.0,0.300752
4,268.0,201.0,287.0,0.299652
5,298.0,224.0,320.0,0.300000
6,338.0,254.0,363.0,0.300275
7,420.0,315.0,450.0,0.300000
8,498.0,374.0,534.0,0.299625
9,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [173]:
df = df_product[['unit_price', 'unit_cost']].dropna()
df['new_price'] = np.ceil(df['unit_cost']/0.7)
df['profit_rate'] = (df['new_price'] - df['unit_cost']) / df['new_price']
df.head(10)

,unit_price,unit_cost,new_price,profit_rate
0,198.0,149.0,213.0,0.300469
1,218.0,164.0,235.0,0.302128
2,230.0,173.0,248.0,0.302419
3,248.0,186.0,266.0,0.300752
4,268.0,201.0,288.0,0.302083
5,298.0,224.0,320.0,0.300000
6,338.0,254.0,363.0,0.300275
7,420.0,315.0,451.0,0.301552
8,498.0,374.0,535.0,0.300935
9,580.0,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [176]:
df = df_product.copy()
df['price_tax'] = np.floor(df['unit_price'] * 1.1)
df.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,price_tax
0,P040101001,04,0401,040101,198.0,149.0,217.0
1,P040101002,04,0401,040101,218.0,164.0,239.0
2,P040101003,04,0401,040101,230.0,173.0,253.0
3,P040101004,04,0401,040101,248.0,186.0,272.0
4,P040101005,04,0401,040101,268.0,201.0,294.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [190]:
df = pd.merge(df_receipt, df_product, how='inner', on='product_cd') \
[['customer_id', 'amount', 'product_cd', 'category_major_cd']]

df_result = df.groupby('customer_id').sum().reset_index()
df_result_07 = df.query("category_major_cd == '07'").groupby('customer_id').sum().reset_index()

df_result = pd.merge(df_result, df_result_07, how='right', on='customer_id')
df_result.columns = ['customer_id', 'amount', 'amount_07']
df_result['ratio'] = df_result['amount_07'] / df_result['amount']
df_result.head(10)

,customer_id,amount,amount_07,ratio
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [224]:
df = pd.merge(df_receipt, df_customer, how='inner', on='customer_id')[['customer_id', 'sales_ymd', 'application_date']].drop_duplicates()
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'], format='%Y%m%d')
df['application_date'] = pd.to_datetime(df['application_date'], format='%Y%m%d')
df['days'] = df['sales_ymd'] - df['application_date']
df.head(10)

,customer_id,sales_ymd,application_date,days
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
4,CS006214000001,2018-10-28,2015-02-01,1365 days
7,CS006214000001,2019-09-08,2015-02-01,1680 days
8,CS006214000001,2018-01-31,2015-02-01,1095 days
9,CS006214000001,2017-07-05,2015-02-01,885 days
10,CS006214000001,2018-11-10,2015-02-01,1378 days
12,CS006214000001,2019-04-10,2015-02-01,1529 days
15,CS006214000001,2019-06-01,2015-02-01,1581 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [225]:
df = pd.merge(df_receipt, df_customer, how='inner', on='customer_id')[['customer_id', 'sales_ymd', 'application_date']].drop_duplicates()
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'], format='%Y%m%d')
df['application_date'] = pd.to_datetime(df['application_date'], format='%Y%m%d')
df['months'] = df[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0],x[1]).years*12 + relativedelta(x[0],x[1]).months, axis=1)
df.head(10)

,customer_id,sales_ymd,application_date,months
0,CS006214000001,2018-11-03,2015-02-01,45
1,CS006214000001,2017-05-09,2015-02-01,27
2,CS006214000001,2017-06-08,2015-02-01,28
4,CS006214000001,2018-10-28,2015-02-01,44
7,CS006214000001,2019-09-08,2015-02-01,55
8,CS006214000001,2018-01-31,2015-02-01,35
9,CS006214000001,2017-07-05,2015-02-01,29
10,CS006214000001,2018-11-10,2015-02-01,45
12,CS006214000001,2019-04-10,2015-02-01,50
15,CS006214000001,2019-06-01,2015-02-01,52


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [227]:
df = pd.merge(df_receipt, df_customer, how='inner', on='customer_id')[['customer_id', 'sales_ymd', 'application_date']].drop_duplicates()
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'], format='%Y%m%d')
df['application_date'] = pd.to_datetime(df['application_date'], format='%Y%m%d')
df['years'] = df[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0],x[1]).years, axis=1)
df.head(10)

,customer_id,sales_ymd,application_date,years
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2
10,CS006214000001,2018-11-10,2015-02-01,3
12,CS006214000001,2019-04-10,2015-02-01,4
15,CS006214000001,2019-06-01,2015-02-01,4


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [233]:
df = pd.merge(df_receipt, df_customer, how='inner', on='customer_id')[['customer_id', 'sales_ymd', 'application_date']].drop_duplicates()
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype('str'))
df['application_date'] = pd.to_datetime(df['application_date'])
df['eps'] = (df['sales_ymd'].astype(int) / 10**9) - (df['application_date'].astype(int) / 10**9)
df.head(10)

,customer_id,sales_ymd,application_date,eps
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
4,CS006214000001,2018-10-28,2015-02-01,117936000.0
7,CS006214000001,2019-09-08,2015-02-01,145152000.0
8,CS006214000001,2018-01-31,2015-02-01,94608000.0
9,CS006214000001,2017-07-05,2015-02-01,76464000.0
10,CS006214000001,2018-11-10,2015-02-01,119059200.0
12,CS006214000001,2019-04-10,2015-02-01,132105600.0
15,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [243]:
df = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype(str))
df['weekday'] = df['sales_ymd'].dt.weekday
df['monday'] = df['sales_ymd'] - pd.to_timedelta(df['weekday'], unit='D')
df.head(10)

,customer_id,sales_ymd,weekday,monday
0,CS006214000001,2018-11-03,5,2018-10-29
1,CS008415000097,2018-11-18,6,2018-11-12
2,CS028414000014,2017-07-12,2,2017-07-10
3,ZZ000000000000,2019-02-05,1,2019-02-04
4,CS025415000050,2018-08-21,1,2018-08-20
5,CS003515000195,2019-06-05,2,2019-06-03
6,CS024514000042,2018-12-05,2,2018-12-03
7,CS040415000178,2019-09-22,6,2019-09-16
8,ZZ000000000000,2017-05-04,3,2017-05-01
9,CS027514000015,2019-10-10,3,2019-10-07


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [244]:
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
14610,CS001512000360,佐久間 翔子,1,女性,1961-06-11,57,210-0831,神奈川県川崎市川崎区観音**********,S13001,20160912,0-00000000-0
9482,CS038315000136,小柳 美菜,1,女性,1983-06-12,35,134-0085,東京都江戸川区南葛西**********,S13038,20160106,9-20100917-9
19096,CS002311000127,豊島 咲,1,女性,1980-04-26,38,187-0002,東京都小平市花小金井**********,S13002,20181009,0-00000000-0
13405,CS008414000051,堀井 綾,1,女性,1970-03-17,49,157-0073,東京都世田谷区砧**********,S13008,20150526,B-20100930-E
4733,CS040413000099,蒼井 璃奈子,1,女性,1974-08-27,44,226-0026,神奈川県横浜市緑区長津田町**********,S14040,20150221,0-00000000-0
3880,CS015404000022,戎 博明,0,男性,1969-08-15,49,136-0073,東京都江東区北砂**********,S13015,20150526,0-00000000-0
14332,CS040213000036,奥村 龍吉,9,不明,1989-10-03,29,226-0018,神奈川県横浜市緑区長津田みなみ台**********,S14040,20151214,0-00000000-0
14433,CS021414000065,長尾 恭子,1,女性,1977-11-26,41,259-1144,神奈川県伊勢原市池端**********,S14021,20150924,B-20100416-A
6720,CS006512000218,森山 路子,1,女性,1966-01-06,53,224-0026,神奈川県横浜市都筑区南山田町**********,S14006,20180716,0-00000000-0
12429,CS007212000020,神野 陽子,1,女性,1994-09-24,24,262-0002,千葉県千葉市花見川区内山町**********,S12007,20150526,0-00000000-0


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [248]:
_, df = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender_cd'])
df.groupby('gender_cd').count().reset_index()[['gender_cd', 'customer_id']]

,gender_cd,customer_id
0,0,298
1,1,1793
2,9,107


---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [257]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine='python') \
.groupby('customer_id').sum().reset_index()[['customer_id', 'amount']]
bb_mean = df['amount'].mean()
bb_std = df['amount'].std()
df[abs(df['amount']-bb_mean) >= bb_std*3].head(10)

,customer_id,amount
332,CS001605000009,18925
1755,CS006415000147,12723
1817,CS006515000023,18372
1833,CS006515000125,12575
1841,CS006515000209,11373
1870,CS007115000006,11528
1941,CS007514000056,13293
1943,CS007514000094,15735
1951,CS007515000107,11188
1997,CS007615000026,11959


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [269]:
df = df_receipt.query("not customer_id.str.startswith('Z')", engine='python') \
.groupby('customer_id').sum().reset_index()[['customer_id', 'amount']]
q1, q3 = float(df.quantile(0.25)), float(df.quantile(0.75))
iqr = q3 - q1
df[(df['amount'] < q1-iqr*1.5) | (df['amount'] > q3+iqr*1.5)].head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [270]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [271]:
df_product_1 = df_product.dropna()
len(df_product_1) == len(df_product) - 7
# 正確にはnanが2つのカラムで同一の列であるとは限らない

True

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [272]:
df_product_2 = df_product.fillna(np.round(df_product.mean()))
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [273]:
df_product_3 = df_product.fillna(np.round(df_product.median()))
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [296]:
df_groupby = df_product[['unit_price', 'unit_cost', 'category_small_cd']].groupby('category_small_cd').agg(np.nanmedian).reset_index()
df_groupby.columns = ['category_small_cd', 'unit_price_med', 'unit_cost_med']

df_product_4 = pd.merge(df_product, df_groupby, on='category_small_cd', how='left')
ind = df_product_4.index[df_product_4['unit_price'].isnull()]
df_product_4.loc[ind, 'unit_price'] = df_product_4.loc[ind, 'unit_price_med']
ind = df_product_4.index[df_product_4['unit_cost'].isnull()]
df_product_4.loc[ind, 'unit_cost'] = df_product_4.loc[ind, 'unit_cost_med']
df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
unit_price_med        0
unit_cost_med         0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [309]:
df = pd.merge(df_customer, df_receipt, on='customer_id', how='left')[['customer_id', 'amount', 'sales_ymd']]
df_sum = df.groupby('customer_id').sum().reset_index()
df_2019 = df.query('20190101 <= sales_ymd <= 20191231') \
.groupby('customer_id').sum().reset_index() \
.rename(columns={'amount': 'amount_2019'})

df = pd.merge(df_sum[['customer_id', 'amount']], df_2019[['customer_id', 'amount_2019']],
              on='customer_id', how='left') \
     .fillna(0).query('amount_2019 > 0')
df['rate'] = df['amount_2019'] / df['amount']
df.head(10)

,customer_id,amount,amount_2019,rate
8,CS001113000004,1298.0,1298.0,1.000000
10,CS001114000005,626.0,188.0,0.300319
12,CS001115000010,3044.0,578.0,0.189882
17,CS001205000004,1988.0,702.0,0.353119
18,CS001205000006,3337.0,486.0,0.145640
23,CS001211000025,456.0,456.0,1.000000
30,CS001212000070,456.0,456.0,1.000000
57,CS001214000009,4685.0,664.0,0.141729
59,CS001214000017,4132.0,2962.0,0.716844
61,CS001214000048,2374.0,1889.0,0.795703


---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [312]:
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']],
                         df_geocode[['postal_cd', 'longitude' ,'latitude']],
                         how='left', on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id') \
.agg({'longitude':'mean', 'latitude':'mean'}).reset_index() \
.rename(columns={'longitude':'mean_longitude', 'latitude':'mean_latitude'})

df_customer_1 = pd.merge(df_customer, df_customer_1, how='left', on='customer_id')
df_customer_1.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,mean_longitude,mean_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.68965,35.67374
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.48360,35.39125
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.70775,35.54084


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [321]:
def calc_l(lat1, lon1, lat2, lon2):
    return 6371 * np.arccos(np.sin(np.radians(lat1))*np.sin(np.radians(lat2)) + 
                            np.cos(np.radians(lat1))*np.cos(np.radians(lat2)) * 
                            np.cos(np.radians(lon1)-np.radians(lon2)))

df_customer_1 = df_customer_1.rename(columns={'application_store_cd': 'store_cd'})
df = pd.merge(df_customer_1, df_store, on='store_cd', how='left')
df['distance'] = calc_l(df['mean_latitude'], df['mean_longitude'], df['latitude'], df['longitude'])
df[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.451182
2,CS031415000172,東京都渋谷区代々木**********,東京都渋谷区初台二丁目,0.411733
3,CS028811000001,神奈川県横浜市泉区和泉町**********,神奈川県横浜市瀬谷区二ツ橋町,8.065196
4,CS001215000145,東京都大田区仲六郷**********,東京都大田区仲六郷二丁目,1.268421
5,CS020401000016,東京都板橋区若木**********,東京都北区十条仲原三丁目,4.185905
6,CS015414000103,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673
7,CS029403000008,千葉県浦安市海楽**********,千葉県浦安市東野一丁目,0.804858
8,CS015804000004,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673
9,CS033513000180,神奈川県横浜市旭区善部町**********,神奈川県横浜市瀬谷区阿久和西一丁目,1.956947


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [322]:
df = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_customer_u = pd.merge(df_customer, df, how='left', on='customer_id') \
                .sort_values(['amount', 'customer_id'], ascending=[False, True])
df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first', inplace=True)
df_customer_u.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount
16905,CS017415000097,福士 千夏,1,女性,1973-04-03,45,166-0014,東京都杉並区松ノ木**********,S13017,20151209,F-20101006-F,23086.0
12692,CS015415000185,岩淵 はるみ,1,女性,1973-09-19,45,135-0043,東京都江東区塩浜**********,S13015,20150322,F-20101014-F,20153.0
13550,CS031414000051,長澤 沙知絵,1,女性,1973-04-25,45,151-0064,東京都渋谷区上原**********,S13031,20150823,F-20101009-F,19202.0
4808,CS028415000007,紺野 あい,1,女性,1969-07-28,49,246-0023,神奈川県横浜市瀬谷区阿久和東**********,S14028,20151212,F-20100922-F,19127.0
14205,CS001605000009,安部 耕司,0,男性,1952-10-22,66,144-0035,東京都大田区南蒲田**********,S13001,20160203,F-20101019-E,18925.0


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [323]:
df_customer_n = pd.merge(df_customer, df_customer_u, how='inner', on=['customer_name', 'postal_cd']) \
.rename(columns={'cusomer_id_x': 'customer_id', 'customer_id_y': 'customer_id_u'})
df_customer_n.head()

,customer_id_x,customer_name,gender_cd_x,gender_x,birth_day_x,age_x,postal_cd,address_x,application_store_cd_x,application_date_x,...,customer_id_u,gender_cd_y,gender_y,birth_day_y,age_y,address_y,application_store_cd_y,application_date_y,status_cd_y,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,...,CS021313000114,1,女性,1981-04-29,37,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,...,CS037613000071,9,不明,1952-04-01,66,東京都江東区南砂**********,S13037,20150414,0-00000000-0,NaN
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,...,CS031415000172,1,女性,1976-10-04,42,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,5088.0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,...,CS028811000001,1,女性,1933-03-27,86,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,NaN
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,...,CS001215000145,1,女性,1995-03-29,24,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,875.0


---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [324]:
del df_customer_1, df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [325]:
df = pd.merge(df_customer, df_receipt['customer_id'], on='customer_id', how='inner')
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
len(df_train), len(df_test)

(52545, 13137)

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [340]:
df = df_receipt.copy()
df['sales_ym'] = pd.to_datetime(df['sales_ymd'].astype(str)).dt.strftime('%Y%m')
df = df.groupby('sales_ym').sum().reset_index().sort_values('sales_ym')

datasets = []
for i in range(3):
    datasets.append((df.iloc[i+0:i+12], df.iloc[i+12:i+12+6]))
print(len(datasets[0][0]), len(datasets[0][1]))

12 6


---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [358]:
df = pd.merge(df_customer, df_receipt.groupby('customer_id').sum().reset_index(),
              on='customer_id', how='left')
df['has_history'] = (~df['amount'].isnull()).astype(int)
print(df['has_history'].value_counts())

rus = RandomUnderSampler(random_state=42)
df_0, _ = rus.fit_resample(df, df['has_history'])
print(df_0['has_history'].value_counts())

0    13665
1     8306
Name: has_history, dtype: int64
1    8306
0    8306
Name: has_history, dtype: int64


---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [361]:
df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_ = df_customer.drop('gender', axis=1)

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [366]:
df = df_product.merge(df_category[['category_small_cd', 'category_major_name', 'category_medium_name', 'category_small_name']],
                      how='left', on='category_small_cd')
df.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,04,0401,040101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,04,0401,040101,268.0,201.0,惣菜,御飯類,弁当類


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [367]:
df.to_csv('./data/df_q94.csv', header=True, encoding='utf8', index=False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [368]:
df.to_csv('./data/df_q95.csv', header=True, encoding='cp932', index=False)

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [369]:
df.to_csv('./data/df_q96.csv', header=False, encoding='utf8', index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [371]:
df = pd.read_csv('./data/df_q94.csv')
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [372]:
df = pd.read_csv('./data/df_q96.csv', header=None)
df.head(10)

,0,1,2,3,4,5,6,7,8
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [373]:
df = pd.read_csv('./data/df_q94.csv')
df.to_csv('./data/df_q99.csv', sep='\t', index=False)

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [374]:
df = pd.read_csv('./data/df_q99.csv', sep='\t')
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！

In [375]:
# 上で作成したファイルを削除
!rm ./data/df_q*.csv